# Intermediate Neural Network in TensorFlow

In this notebook, we convert our [intermediate-depth neural network](https://github.com/the-deep-learners/TensorFlow-LiveLessons/blob/master/notebooks/intermediate_net_in_keras.ipynb) from Keras to TensorFlow.